In [3]:
import logging
from extract import *
from transform import *
from load import * 

from datetime import datetime




###########
# Extract #
###########



# Connect to database
conn = connect_db('staging_disasters')
if conn:
    cursor = conn.cursor()

    # Process CSV file and load data into staging table
    process_csv_to_staging(cursor, "staging_disasters", r"C:\Users\Legion\Desktop\official_data\1900_2021_DISASTERS.csv")

    conn.commit()
    cursor.close()
    conn.close()





#############
# Transform #
#############



disasters = get_data_from_db(f""" --sql
        SELECT * FROM staging_disasters 
        WHERE TO_CHAR(extraction_time, 'YYYYMMDD') = '{datetime.now().strftime("%Y%m%d")}';
    """, 'staging_disasters')

columns_to_remove = ['Local Time', 'River Basin', 'Admin1 Code', 'Admin2 Code', 'Geo Locations']
if disasters is not None:
    transformed_disasters = transform_data(disasters, columns_to_remove)


########
# Load #
########
create_disaster_tables(connect_db('disasters_dwh'))

fact_disasters, dim_disaster_types, dim_disaster_groups, dim_associated_distructions, dim_locations, dim_disaster_names, dim_ofda_responses, dim_appeals, dim_declarations, dim_mag_scales, dim_adm_levels,dim_disasters_origin,dim_dates=generate_dimensions(transformed_disasters)




c:\Users\Legion\Desktop\final-project-FSD\etl_pipeline\transform.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
ERROR:root:Error creating dim_disaster_groups: syntax error at or near "CREATE"
LINE 2:             CREATE TABLE IF NOT EXISTS dim_disaster_groups (
                    ^

ERROR:root:Error creating dim_disaster_types: can't call .execute() on named cursors more than once
ERROR:root:Error creating dim_disaster_names: can't call .execute() on named cursors more than once
ERROR:root:Error creating dim_locations: can't call .execute() on named cursors more than once
ERROR:root:Error creating dim_dates: can't call .execute() on named cursors more than once
ERROR:root:Error creating dim_associated_distructions: can't call .execute() on named cursors more than once
ERROR:root:Error creat

ProgrammingError: named cursor isn't valid anymore